In [1]:
import json
import numpy as np
from PIL import Image, ImageDraw

def initialize_image_and_z_buffer(canvas_size):
    image = Image.new('RGB', (canvas_size, canvas_size), (0, 0, 0))
    z_buffer = np.full((canvas_size, canvas_size), np.inf, dtype=float)
    return image, z_buffer

def perspective_projection_matrix(near, far, left, right, top, bottom):
    matrix = np.zeros((4, 4))
    matrix[0, 0] = 2 * near / (right - left)
    matrix[1, 1] = 2 * near / (top - bottom)
    matrix[0, 2] = (right + left) / (right - left)
    matrix[1, 2] = (top + bottom) / (top - bottom)
    matrix[2, 2] = -(far + near) / (far - near)
    matrix[2, 3] = -2 * far * near / (far - near)
    matrix[3, 2] = -1
    return matrix

def camera_matrix(u, v, n, r):
    matrix = np.eye(4)
    matrix[:3, 0] = u
    matrix[:3, 1] = v
    matrix[:3, 2] = n
    matrix[:3, 3] = -r
    matrix[0, 3] = -np.dot(r, u)
    matrix[1, 3] = -np.dot(r, v)
    matrix[2, 3] = -np.dot(r, n)
    matrix[3, 3] = 1
    return matrix

def line_eq(v0, v1, x, y):
    return (v0[1] - v1[1]) * x + (v1[0] - v0[0]) * y + (v0[0] * v1[1] - v1[0] * v0[1])
    
def compute_triangle_color(normal):
    dotp = 0.707 * normal[0] + 0.5 * normal[1] + 0.5 * normal[2]
    dotp = abs(dotp) if dotp < 0 else (1.0 if dotp > 1.0 else dotp)
    tri_color = [int(255 * 0.95 * dotp), int(255 * 0.65 * dotp), int(255 * 0.88 * dotp)]
    return tuple(tri_color)

def clip_and_clamp_coordinates(v0, v1, v2, canvas_width, canvas_height):
    xmin, xmax = max(np.floor(min(v0[0], v1[0], v2[0])), 0), min(np.ceil(max(v0[0], v1[0], v2[0])), canvas_width - 1)
    ymin, ymax = max(np.floor(min(v0[1], v1[1], v2[1])), 0), min(np.ceil(max(v0[1], v1[1], v2[1])), canvas_height - 1)
    return int(xmin), int(xmax), int(ymin), int(ymax)
    
def render_triangle(triangle, draw, z_buffer, raster_vertex):
    v0, v1, v2 = raster_vertex[0], raster_vertex[1], raster_vertex[2]
    normal = np.array(triangle["v0"]["n"])
    color = compute_triangle_color(normal)

    canvas_width, canvas_height = draw.im.size

    xmin, xmax, ymin, ymax = clip_and_clamp_coordinates(v0, v1, v2, canvas_width, canvas_height)

    f12_v0 = line_eq(v1, v2, v0[0], v0[1])
    f20_v1 = line_eq(v2, v0, v1[0], v1[1])
    f01_v2 = line_eq(v0, v1, v2[0], v2[1])

    for y in range(ymin, ymax + 1):
        for x in range(xmin, xmax + 1):
            alpha = line_eq(v1, v2, x, y) / f12_v0
            beta = line_eq(v2, v0, x, y) / f20_v1
            gamma = line_eq(v0, v1, x, y) / f01_v2

            if 0 <= alpha <= 1 and 0 <= beta <= 1 and 0 <= gamma <= 1:
                z = alpha * v0[2] + beta * v1[2] + gamma * v2[2]
                if z < z_buffer[y, x]:
                    draw.point((x, y), fill=color)
                    z_buffer[y, x] = z

# Constants
canvas_size = 256
near, far = 2, 4
left, right, top, bottom = -10, 10, 10, -10

# Load teapot data from JSON
with open('teapot.json', 'r') as f:
    teapot_data = json.load(f)

# Initialize image and z-buffer
image, z_buffer = initialize_image_and_z_buffer(canvas_size)

draw = ImageDraw.Draw(image)
image_width, image_height = draw.im.size

# Define the camera space axes
u = np.array([1, 0, 0])
v = np.array([0, 1, 0])
n = np.array([0, 0, 1])
r = np.array([0, 0, 20])

print(u,v,n,r)

# Create the camera matrix
camera_mat = camera_matrix(u, v, n, r)
print(camera_mat)

# Create perspective projection matrix
perspective_mat = perspective_projection_matrix(near, far, left, right, top, bottom)
print(perspective_mat)

vertices = []
raster_vertices = []

# Iterate over each triangle in the teapot data
for triangle in teapot_data['data']:
    raster_vertex = []
    for vertex in [triangle['v0'], triangle['v1'], triangle['v2']]:
        v = np.array(vertex['v'][:3] + [1.0])  # Add 1.0 for 4D
        v_camera = camera_mat @ v
        v_ndc = v_camera[:3] / v_camera[3]
        v_ndc = np.append(v_ndc, 1)
        v_perspective = perspective_mat @ v_ndc
        vertices.append(v_perspective)

        # Convert to raster coordinates
        x = (v_perspective[0] + 1) * ((image_width - 1) / 2)
        y = (1 - v_perspective[1]) * ((image_height - 1) / 2)
        z = v_perspective[2]
        raster_vertex.append([x, y, z])
    raster_vertices.append(raster_vertex)

print(raster_vertices)
# Iterate over each triangle in the teapot data and raster vertices together
for triangle, raster_vertex in zip(teapot_data['data'], raster_vertices):
    render_triangle(triangle, draw, z_buffer, raster_vertex)

# Save the rendered image
image.save('teapot_rendered.png')

[1 0 0] [0 1 0] [0 0 1] [ 0  0 20]
[[  1.   0.   0.   0.]
 [  0.   1.   0.   0.]
 [  0.   0.   1. -20.]
 [  0.   0.   0.   1.]]
[[ 0.2  0.   0.   0. ]
 [ 0.   0.2  0.   0. ]
 [ 0.   0.  -3.  -8. ]
 [ 0.   0.  -1.   0. ]]
[[[163.20000000000002, 70.125, 52.0], [160.43325, 70.125, 50.3515], [159.973791, 68.242386, 50.374497999999996]], [[163.20000000000002, 70.125, 52.0], [159.973791, 68.242386, 50.374497999999996], [162.7019595, 68.242386, 52.0]], [[162.7019595, 68.242386, 52.0], [159.973791, 68.242386, 50.374497999999996], [160.5067665, 67.614831, 50.347819]], [[162.7019595, 68.242386, 52.0], [160.5067665, 67.614831, 50.347819], [163.27968750000002, 67.6148565, 52.0]], [[163.27968750000002, 67.6148565, 52.0], [160.5067665, 67.614831, 50.347819], [161.591052, 68.242386, 50.293546]], [[163.27968750000002, 67.6148565, 52.0], [161.591052, 68.242386, 50.293546], [164.4550845, 68.242386, 52.0]], [[164.4550845, 68.242386, 52.0], [161.591052, 68.242386, 50.293546], [162.785625, 70.125, 50.23375